Title

In [22]:
import sys
import os
import pandas as pd
from data_utils import load

In [23]:
print("-")


NameError: name 'openai' is not defined

In [16]:
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import torch
print("-")

-


In [11]:
import psutil

# Total RAM in GB
total_ram = psutil.virtual_memory().total / (1024**3)
print(f"Total RAM: {total_ram:.2f} GB")

Total RAM: 31.43 GB


In [22]:
from transformers import BlipProcessor, BlipForConditionalGeneration

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

In [ ]:
from transformers import BlipProcessor, BlipForConditionalGeneration

models = [
    "Salesforce/blip-image-captioning-base",
    "Salesforce/blip2-flan-t5-xl",
    "Salesforce/blip2-opt-2.7b"
]

for name in models:
    try:
        print(f"Testing {name}...")
        processor = BlipProcessor.from_pretrained(name, local_files_only=True)
        model = BlipForConditionalGeneration.from_pretrained(name, local_files_only=True)
        print(f" {name} loaded successfully.\n")
    except Exception as e:
        print(f" {name} failed: {e}\n")

In [ ]:
from transformers import Blip2Processor, Blip2ForConditionalGeneration
from PIL import Image
import torch

# 1️⃣ Load processor and model
processor = Blip2Processor.from_pretrained("Salesforce/blip2-flan-t5-xl")
model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-flan-t5-xl")

# Select device (CPU in this case)
device = "cpu"
model.to(device)

# Load an image
img_path = "data/Images_test-20240424T090736Z-001/Images_test/1.jpg"
image = Image.open(img_path).convert("RGB")

# Prepare inputs
prompt = "Identify tobacco brands in this picture."
inputs = processor(images=image, text=prompt, return_tensors="pt").to(device)

# Generate caption
with torch.no_grad():
    outputs = model.generate(**inputs, max_new_tokens=50)

# Decode and print caption
caption = processor.decode(outputs[0], skip_special_tokens=True)
print("Generated caption:", caption)

In [ ]:
from transformers import Blip2Processor, Blip2ForConditionalGeneration
from PIL import Image
import torch

# Load BLIP-2 small (faster, less memory)
processor = Blip2Processor.from_pretrained("Salesforce/blip2-flan-t5-xl")
model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-flan-t5-xl")

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Load an image
img_path = "data/Images_test-20240424T090736Z-001/Images_test/2.jpg"
image = Image.open(img_path).convert("RGB")

# Generate caption
prompt = "Describe any tobacco products in this image."
inputs = processor(images=image, text=prompt, return_tensors="pt").to(device)

with torch.no_grad():
    outputs = model.generate(**inputs)

caption = processor.decode(outputs[0], skip_special_tokens=True)
print("Caption:", caption)

In [ ]:
import torch
from PIL import Image
from transformers import AutoProcessor, AutoModelForZeroShotObjectDetection, infer_device

model_id = "IDEA-Research/grounding-dino-tiny"
device = infer_device()

processor = AutoProcessor.from_pretrained(model_id)
model = AutoModelForZeroShotObjectDetection.from_pretrained(model_id).to(device)

# Load an image
img_path = "data/Images_test-20240424T090736Z-001/Images_test/2.jpg"
image = Image.open(img_path).convert("RGB")
text_labels = [["a cat", "a remote control"]]

inputs = processor(images=image, text=text_labels, return_tensors="pt").to(model.device)
with torch.no_grad():
    outputs = model(**inputs)

In [ ]:
from transformers import AutoImageProcessor, DetaForObjectDetection
from PIL import Image
import requests

img_path = "data/Images_test-20240424T090736Z-001/Images_test/2.jpg"
image = Image.open(img_path).convert("RGB")

image_processor = AutoImageProcessor.from_pretrained("jozhang97/deta-swin-large")
model = DetaForObjectDetection.from_pretrained("jozhang97/deta-swin-large")

inputs = image_processor(images=image, return_tensors="pt")
outputs = model(**inputs)

# convert outputs (bounding boxes and class logits) to Pascal VOC format (xmin, ymin, xmax, ymax)
target_sizes = torch.tensor([image.size[::-1]])
results = image_processor.post_process_object_detection(outputs, threshold=0.2, target_sizes=target_sizes)[
    0
]
for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
    box = [round(i, 2) for i in box.tolist()]
    print(
        f"Detected {model.config.id2label[label.item()]} with confidence "
        f"{round(score.item(), 3)} at location {box}"
    )

In [ ]:
from transformers import pipeline
from PIL import Image
import requests

# Load the zero-shot object detection pipeline with the specified model
detector = pipeline("zero-shot-object-detection", model="iSEE-Laboratory/llmdet_large")

# Define the candidate classes (tobacco-related items)
candidate_classes = ["cigarette", "vape", "lighter", "cigar", "tobacco pouch","smoke"]

# Load an example image
img_path = "data/Images_test-20240424T090736Z-001/Images_test/2.jpg"
image = Image.open(img_path).convert("RGB")

# Perform zero-shot object detection
results = detector(image, candidate_labels=candidate_classes, threshold=0.5)

print("Detected objects:")
for result in results:
    print(f"Label: {result['label']}, Score: {result['score']:.3f}")

In [ ]:
print("Detected objects:")
print(results)
for result in results:
    print(f"Label: {result['label']}, Score: {result['score']:.3f}")
print(image.size) 